In [40]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd
import rasterio as rio
from scipy.stats import gaussian_kde
import seaborn as sns
import numpy as np

In [2]:

df = pd.read_csv( r'/Volumes/PhD/imagery/masters/output/hyper_compare/MvH_results.csv')
df


,BASIN_ID,RI,M1M,M1Md,M1PF,M1Q1,HM1M,HM1Md,HM1PF,HM1Q1,OHM1M,OHM1Md,OHM1PF,OHM1Q1
0,12042,12.0,0.064260,0.053244,0.046768,0.045561,0.070095,0.055214,0.047952,0.045286,0.069559,0.052435,0.047878,0.041563
1,12042,16.0,0.086179,0.067387,0.056811,0.054883,0.096479,0.070677,0.058715,0.054447,0.095525,0.066046,0.058596,0.048609
2,12042,20.0,0.114659,0.084950,0.068855,0.065981,0.131391,0.090059,0.071712,0.065334,0.129830,0.082879,0.071532,0.056778
3,12042,24.0,0.150996,0.106566,0.083226,0.079135,0.176468,0.114104,0.087318,0.078218,0.174084,0.103528,0.087061,0.066224
4,12042,28.0,0.196295,0.132885,0.100274,0.094646,0.232865,0.143554,0.105934,0.093389,0.229444,0.128600,0.105577,0.077114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,22668,24.0,0.217452,0.275691,0.357919,0.095945,0.254794,0.340418,0.417101,0.132851,0.258005,0.354206,0.417102,0.104975
372,22668,28.0,0.291314,0.372401,0.480807,0.117953,0.343649,0.458428,0.553428,0.170279,0.348095,0.476093,0.553430,0.130639
373,22668,32.0,0.378143,0.480530,0.606062,0.144203,0.444988,0.581294,0.682167,0.215631,0.450542,0.600896,0.682169,0.161446
374,22668,36.0,0.473558,0.590514,0.718774,0.175134,0.551118,0.694841,0.788010,0.269142,0.557361,0.713838,0.788011,0.197864


In [33]:
# change so all basins are included
   

basin = df[df['BASIN_ID']==12042]
basin_id = basin['BASIN_ID'].values[0]

basin

,BASIN_ID,RI,M1M,M1Md,M1PF,M1Q1,HM1M,HM1Md,HM1PF,HM1Q1,OHM1M,OHM1Md,OHM1PF,OHM1Q1
0,12042,12.0,0.064260,0.053244,0.046768,0.045561,0.070095,0.055214,0.047952,0.045286,0.069559,0.052435,0.047878,0.041563
1,12042,16.0,0.086179,0.067387,0.056811,0.054883,0.096479,0.070677,0.058715,0.054447,0.095525,0.066046,0.058596,0.048609
2,12042,20.0,0.114659,0.084950,0.068855,0.065981,0.131391,0.090059,0.071712,0.065334,0.129830,0.082879,0.071532,0.056778
3,12042,24.0,0.150996,0.106566,0.083226,0.079135,0.176468,0.114104,0.087318,0.078218,0.174084,0.103528,0.087061,0.066224
4,12042,28.0,0.196295,0.132885,0.100274,0.094646,0.232865,0.143554,0.105934,0.093389,0.229444,0.128600,0.105577,0.077114
5,12042,32.0,0.251164,0.164506,0.120356,0.112825,0.300704,0.179070,0.127962,0.111147,0.296099,0.158669,0.127481,0.089623
6,12042,36.0,0.315351,0.201901,0.143817,0.133979,0.378554,0.221105,0.153782,0.131792,0.372750,0.194202,0.153151,0.103932
7,12042,40.0,0.387456,0.245299,0.170962,0.158390,0.463208,0.269763,0.183715,0.155599,0.456379,0.235465,0.182907,0.120224


In [67]:
# if values do not cross .5, predict the value at .5
def find_crossing(x, y, threshold=0.5, extrapolate=False):
    for i in range(1, len(x)):
        if y[i-1] < threshold <= y[i]:
            # Interpolation
            x0, x1 = x[i-1], x[i]
            y0, y1 = y[i-1], y[i]
            return x0 + (threshold - y0) * (x1 - x0) / (y1 - y0)
    
    # If threshold not crossed and extrapolation is allowed
    if extrapolate and len(x) >= 2:
        # Use last two points to extrapolate forward
        x0, x1 = x[-2], x[-1]
        y0, y1 = y[-2], y[-1]
        
        # Only extrapolate if y is increasing
        if y1 > y0:
            slope = (y1 - y0) / (x1 - x0)
            return x1 + (threshold - y1) / slope
    
    return np.nan  # No crossing and extrapolation not possible

def estimate_threshold_crossing(x, y, threshold=0.5, degree=2, limit_extrapolation=True):
    x = np.array(x)
    y = np.array(y)
    
    # Remove any NaNs just in case
    mask = ~np.isnan(x) & ~np.isnan(y)
    x = x[mask]
    y = y[mask]

    if len(x) < degree + 1:
        return np.nan  # Not enough points to fit
    
    # Fit polynomial
    coeffs = np.polyfit(x, y, deg=degree)
    poly = np.poly1d(coeffs)

    # Solve poly(x) - threshold = 0
    roots = (poly - threshold).roots
    real_roots = [r.real for r in roots if np.isreal(r)]

    # Filter to extrapolated root that is beyond the max x
    valid_roots = [r for r in real_roots if r > x.max()]

    if limit_extrapolation:
        valid_roots = [r for r in valid_roots if r <= x.max() + 10]  # optional safeguard

    return valid_roots[0] if valid_roots else np.nan

In [61]:
RI = basin['RI'].values
RI

array([12., 16., 20., 24., 28., 32., 36., 40.])

In [62]:
model_columns = [col for col in basin.columns if col not in ['BASIN_ID', 'RI']]
model_columns

['M1M',
 'M1Md',
 'M1PF',
 'M1Q1',
 'HM1M',
 'HM1Md',
 'HM1PF',
 'HM1Q1',
 'OHM1M',
 'OHM1Md',
 'OHM1PF',
 'OHM1Q1']

In [64]:
results = {'BASIN_ID': [basin_id]} 
for col in model_columns:
    results[col] = [find_crossing(RI, basin[col].values)]


results_df = pd.DataFrame(results)
results_df

,BASIN_ID,M1M,M1Md,M1PF,M1Q1,HM1M,HM1Md,HM1PF,HM1Q1,OHM1M,OHM1Md,OHM1PF,OHM1Q1
0,12042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
all_basins = df.groupby('BASIN_ID').apply(
    lambda x: pd.Series({
        col: find_crossing(x['RI'].values, x[col].values, threshold=0.5, extrapolate=True)
        for col in model_columns
    })
).reset_index()

all_basins.columns = ['BASIN_ID'] + model_columns
all_basins

/var/folders/mm/sjj0b1k91z39pkxfzn0rj_1h0000gn/T/ipykernel_74624/1478333958.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_basins = df.groupby('BASIN_ID').apply(


,BASIN_ID,M1M,M1Md,M1PF,M1Q1,HM1M,HM1Md,HM1PF,HM1Q1,OHM1M,OHM1Md,OHM1PF,OHM1Q1
0,12042,46.243422,63.475374,88.486177,95.974905,41.738437,58.926609,82.265976,97.864178,42.086431,65.643824,82.625814,133.242501
1,12057,27.547009,25.790321,23.907717,34.946492,24.435624,23.186334,21.847502,31.133086,23.719794,21.741610,21.874629,31.798587
2,12518,37.896469,52.240237,56.711333,60.235187,35.825316,49.480413,56.131809,60.818240,34.204924,45.864392,56.160557,70.034688
3,12524,21.590789,20.974311,17.636136,26.992071,21.067573,20.629586,17.369329,26.662317,20.245416,19.431655,17.379075,28.634424
4,12527,25.163537,26.049402,37.881699,36.537329,25.602322,26.151851,38.641013,37.523374,25.823867,27.764557,38.671684,39.763299
5,12616,57.762069,74.208276,79.782158,89.232552,49.218846,58.807018,67.895254,82.375716,49.486553,64.251945,67.817537,119.533199
6,12860,23.219485,23.344548,22.987270,27.096467,24.472846,24.739132,25.012136,26.872845,27.548849,27.888785,24.988445,33.392537
7,12997,21.275839,20.872247,20.752247,22.844215,22.149679,22.238264,21.687887,24.909073,22.520502,23.876975,21.693260,27.976555
8,13281,48.564416,79.913784,112.131417,125.859532,42.430292,64.276239,87.265368,103.244138,43.057765,62.974510,87.037970,154.482421
9,17346,22.417783,20.443921,19.434882,28.694806,20.854247,19.111953,17.893938,27.251502,20.787887,18.685763,17.908932,29.477545
